[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/babbu3682/Med_ChatGPT_tutorial/blob/main/Notebook/Hands_On_Inference.ipynb)

In [1]:
cd /workspace/sunggu/7.Mentor/RSNA2023

[Errno 2] No such file or directory: '/workspace/sunggu/7.Mentor/RSNA2023'
/workspace/sunggu/7.Mentor/Med_ChatGPT_tutorial/Notebook


In [2]:
!nvidia-smi
import os
os.environ["CUDA_DEVICE_ORDER"]     =  'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"]  =  '3,4,5,6'
print("CPU 갯수 = ", os.cpu_count())

Mon Jul  3 20:10:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:1B:00.0 Off |                    0 |
| N/A   53C    P0   191W / 250W |  17483MiB / 22919MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:1C:00.0 Off |                    0 |
| N/A   

In [ ]:
# !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117
# !pip freeze > /workspace/sunggu/0.Challenge/requirements.txt
!pip install -r requirements.txt
!pip install -U nibabel

In [ ]:
test_data['path']  = f"{data_dir}/train_image/" + test_data['patient_id'].astype(str) + "/" + test_data['image_id'].astype(str) + ".dcm"

# 0. Fix Seed

In [3]:
import torch
import random
import numpy as np

# fix random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# 1. Dataset

In [4]:
import os
import re
import cv2
import skimage
import pydicom
import pandas as pd
import glob
import albumentations as A
from pydicom.pixel_data_handlers.util import apply_modality_lut, apply_voi_lut
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset

def list_sort_nicely(l):
    def convert(text): return int(text) if text.isdigit() else text
    def alphanum_key(key): return [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

def fixed_clahe(image, **kwargs):
    clahe_mat = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe_mat.apply(image)

def change_to_uint8(image, **kwargs):
    return skimage.util.img_as_ubyte(image)

def change_to_float32(image, **kwargs):
    return skimage.util.img_as_float32(image)

def min_max_normalization(image, **kwargs):
    if len(np.unique(image)) != 1:
        image = image.astype('float32')
        image -= image.min()
        image /= image.max() 
    return image

def get_transforms():
    return A.Compose([
        # Preprocessing
        A.Resize(224, 224),
        A.Lambda(image=min_max_normalization, always_apply=True),
        A.Lambda(image=change_to_uint8, always_apply=True),
        A.Lambda(image=fixed_clahe, always_apply=True),
        A.Lambda(image=change_to_float32, always_apply=True),
        
        # Normalize
        A.Lambda(image=min_max_normalization, always_apply=True),
        A.Normalize(max_pixel_value=1.0, mean=0.5, std=0.5),
        ToTensorV2()
    ])

class RSNA_Dataset(Dataset):
    def __init__(self, mode="test"):
        self.root       = '/workspace/sunggu/0.Challenge/Med_tutorial_ChatGPT/dataset/test_images'
        self.data_df    = pd.read_csv('/workspace/sunggu/0.Challenge/Med_tutorial_ChatGPT/dataset/test.csv')
        self.transforms = get_transforms()
        print(f"len of data: {len(self.data_df)}")

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, str(self.data_df['patient_id'].iloc[idx]), str(self.data_df['image_id'].iloc[idx]) + '.dcm')
        dcm_data = pydicom.dcmread(img_path)
        temp_img = apply_modality_lut(dcm_data.pixel_array, dcm_data)   
        image    = apply_voi_lut(temp_img, dcm_data)                             

        # add channel axis
        image    = np.expand_dims(image, axis=-1)
        image    = self.transforms(image=image)['image']
        
        return image, self.data_df['prediction_id'].iloc[idx]

In [5]:
from torch.utils.data import DataLoader

# 1. Create Dataset
test_dataset = RSNA_Dataset()

# 2. Create DataLoader
test_loader  = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=32)

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/sunggu/0.Challenge/Med_tutorial_ChatGPT/dataset/test.csv'

# 2. Network

In [8]:
import torch.nn as nn
from torchvision import models
from transformers import ViTModel, ViTConfig

class Resnet50(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.model = models.resnet50(pretrained=pretrained)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc = nn.Linear(2048, 1)
        
    def forward(self, x):
        x = self.model(x)
        return x
    
class ViT(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.model = ViTModel(ViTConfig())
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc = nn.Linear(768, 1)
        
    def forward(self, x):
        x = self.model(x)
        return x


In [9]:
model = Resnet50(pretrained=True) # Resnet50
# model = ViT(pretrained=True) # ViT
    
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Number of Learnable Params:', n_parameters)   

/home/sunggu/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sunggu/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of Learnable Params: 23503809


# 3. Load checkpoint

In [85]:
checkpoint_path = '/workspace/sunggu/0.Challenge/Med_tutorial_ChatGPT/checkpoints/230615_ResNet50_L2_Reg'
save_dir        = '/workspace/sunggu/0.Challenge/Med_tutorial_ChatGPT/predictions/230615_ResNet50_L2_Reg'

# make folder if not exist
os.makedirs(save_dir, exist_ok =True)

# Resume
# Metric  =  valid_F1
# Argsort =  [ 7 10 14 12  6]
# Value   =  [0.6940299 0.6783217 0.6756757 0.6748971 0.6641791]

filename = 'epoch_7_checkpoint.pth'
print("=> loading checkpoint '{}'".format(filename))
checkpoint  = torch.load(os.path.join(checkpoint_path, filename))
model.load_state_dict(checkpoint['model_state_dict'])

# using GPU
model = model.to('cuda')

=> loading checkpoint 'epoch_7_checkpoint.pth'


# 4. inference Loop

In [86]:
import time
import datetime
from tqdm import tqdm

In [90]:
@torch.no_grad()
def test_loop_fn(test_loader, model, device):
    model.eval()
    epoch_iterator = tqdm(test_loader, desc="TEST (X / X Steps) (loss=X.X)", dynamic_ncols=True, total=len(test_loader))

    preds = [] # must be 1d list or array
    ids   = [] # must be 1d list or array
    for step, batch_data in enumerate(epoch_iterator):
        image, id = batch_data
        image = image.to(device)

        logit = model(image)
        epoch_iterator.set_description("TEST: (%d / %d Steps)" % (step, len(test_loader)))

        # post-processing
        preds.append(logit.sigmoid().squeeze().detach().cpu().numpy())
        ids.append(id[0])

    # Metric Calculation
    preds = np.array(preds)
    ids   = np.array(ids) 
    
    return preds, ids

In [91]:
print(f"Start Testing")
start_time = time.time()

preds, ids = test_loop_fn(test_loader, model, device='cuda')

# Finish
total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Testing time {}'.format(total_time_str))

Start Testing


TEST: (3 / 4 Steps): 100%|██████████| 4/4 [00:04<00:00,  1.16s/it]  

Testing time 0:00:04


# 5. Submission for Kaggle

In [92]:
preds

array([0.33571672, 0.3146994 , 0.4178922 , 0.5277957 ], dtype=float32)

In [93]:
ids

array(['10008_L', '10008_L', '10008_R', '10008_R'], dtype='<U7')

In [94]:
# read csv
import pandas as pd
df = pd.DataFrame()

df['prediction_id'] = ids
df['cancer'] = preds

In [95]:
df

,prediction_id,cancer
0,10008_L,0.335717
1,10008_L,0.314699
2,10008_R,0.417892
3,10008_R,0.527796


In [96]:
grouped_mean = df.groupby('prediction_id').mean()
grouped_mean 

,cancer
prediction_id,
10008_L,0.325208
10008_R,0.472844


In [97]:
reset_df = grouped_mean.reset_index()

In [98]:
reset_df

,prediction_id,cancer
0,10008_L,0.325208
1,10008_R,0.472844


In [99]:
# save csv
reset_df.to_csv('/workspace/sunggu/0.Challenge/RSNA2023/dataset/rsna-breast-cancer-detection/sample_submission_2.csv', index=False)

Exception ignored in: Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
<function _ConnectionBase.__del__ at 0x7efba56c7040>
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread QueueFeederThread:
Traceback (most recent call last):

OSError: Cannot save file into a non-existent directory: '/workspace/sunggu/0.Challenge/RSNA2023/dataset/rsna-breast-cancer-detection'